In [ ]:
from ultralytics import YOLO
import os
import cv2
import xml.etree.ElementTree as ET

In [ ]:
# Load model (COCO-pretrained YOLOv8)
model = YOLO("yolov8n.pt")

In [ ]:
# Image folder
img_dir = "images/"
out_dir = "annotations/"

os.makedirs(out_dir, exist_ok=True)

for img_file in os.listdir(img_dir):
    if not img_file.endswith(".jpg"):
        continue
    
    img_path = os.path.join(img_dir, img_file)
    results = model(img_path)

    # Get detections
    for r in results:
        h, w, _ = cv2.imread(img_path).shape
        
        # Create XML root
        annotation = ET.Element("annotation")
        ET.SubElement(annotation, "filename").text = img_file

        for box in r.boxes:
            cls = model.names[int(box.cls)]
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            obj = ET.SubElement(annotation, "object")
            ET.SubElement(obj, "name").text = cls
            bndbox = ET.SubElement(obj, "bndbox")
            ET.SubElement(bndbox, "xmin").text = str(x1)
            ET.SubElement(bndbox, "ymin").text = str(y1)
            ET.SubElement(bndbox, "xmax").text = str(x2)
            ET.SubElement(bndbox, "ymax").text = str(y2)

        # Save XML in Pascal VOC format
        tree = ET.ElementTree(annotation)
        tree.write(os.path.join(out_dir, img_file.replace(".jpg", ".xml")))